# Generate Orders from Positions

In [4]:
import pandas as pd
import numpy as np
import io

### Data

In [5]:
def read(s):
    df = pd.read_csv(io.StringIO(s), sep="|") # io.StringIO() here let's us treat the string s as though it were a .csv file, necessary for pandas
    df.rename(columns = lambda s : s.strip(), inplace=True) # anonymous function stripping whitespace
    for col in df.columns:
        if col in ["dt", "rt"]:
            df[col] = pd.to_datetime(df[col]).dt.tz_localize('UTC') # need to localize to UTC to compare with timestamp
        elif df.dtypes[col] == np.dtype("O"): #dtype("0") are non-numeric np objects
            df[col] = df[col].apply(lambda s: s.strip())
        else:
            pass
    return df

1. rt - reference time: when the strategy wants to achieve given target position
2. dt - decision time: when the strategy decided to achieve that position

In [6]:
# Target Positions
tpos = read("""
    rt                   | strategy | asset | dt                   | value
    2023-08-29 07:00:00z | s1       | PLN   | 2023-08-29 07:00:00z | 4e6
    2023-08-29 16:00:00z | s1       | CZK   | 2023-08-29 07:00:00z | -24e6
    2023-08-30 07:00:00z | s1       | PLN   | 2023-08-30 07:00:00z | 8e6
    2023-08-30 16:00:00z | s1       | CZK   | 2023-08-30 07:00:00z | -48e6
    2023-08-29 07:00:00z | s2       | EUR   | 2023-08-29 07:00:00z | 1.8e6
    2023-08-29 16:00:00z | s2       | AUD   | 2023-08-29 07:00:00z | -1.5e6
    2023-08-30 07:00:00z | s2       | EUR   | 2023-08-30 07:00:00z | 3.6e6
    2023-08-30 16:00:00z | s2       | AUD   | 2023-08-30 07:00:00z | -1e6
    2023-08-29 07:00:00z | s3       | CZK   | 2023-08-29 07:00:00z | 12e6
    2023-08-29 16:00:00z | s3       | CZK   | 2023-08-29 07:00:00z | 18e6
    2023-08-30 07:00:00z | s3       | CZK   | 2023-08-30 07:00:00z | 18e6
    2023-08-30 16:00:00z | s3       | CZK   | 2023-08-30 07:00:00z | 24e6
    """)

tpos.rename(columns={'value': 'target_position'}, inplace=True)

In [7]:
fx_rates = read("""
    rt                   | asset | value
    2023-08-29 07:00:00z | PLN   | 3.934
    2023-08-29 16:00:00z | PLN   | 3.924
    2023-08-30 07:00:00z | PLN   | 3.914
    2023-08-30 16:00:00z | PLN   | 3.904
    
    2023-08-29 07:00:00z | CZK   | 23.12
    2023-08-29 16:00:00z | CZK   | 23.08
    2023-08-30 07:00:00z | CZK   | 23.02
    2023-08-30 16:00:00z | CZK   | 23.01
    
    2023-08-29 07:00:00z | EUR   | 1.116
    2023-08-29 16:00:00z | EUR   | 1.119
    2023-08-30 07:00:00z | EUR   | 1.121
    2023-08-30 16:00:00z | EUR   | 1.122
    
    2023-08-29 07:00:00z | AUD   | 0.672
    2023-08-29 16:00:00z | AUD   | 0.682
    2023-08-30 07:00:00z | AUD   | 0.689
    2023-08-30 16:00:00z | AUD   | 0.690
    """)

fx_rates.rename(columns={'value': 'fx_rate'}, inplace=True)

fx_rates.loc[fx_rates['asset'] == "AUD", 'fx_rate'] = 1 / fx_rates.loc[fx_rates['asset'] == "AUD", 'fx_rate']
fx_rates.loc[fx_rates['asset'] == "EUR", 'fx_rate'] = 1 / fx_rates.loc[fx_rates['asset'] == "EUR", 'fx_rate']


In [8]:
min_order_size_usd = read("""
    asset | value
    PLN   | 5e5
    CZK   | 5e5
    EUR   | 1e6
    AUD   | 1e6
""")

min_order_size_usd.rename(columns={'value': 'min_order_size_usd'}, inplace=True)

In [9]:
trading_session = pd.Timestamp("2023-08-30 16:00:00z")

### Generate orders (total & by strategy) as of trading session in local currency

In [10]:
def find_last_position(asset, past_positions, reference_time):
    asset_rows = past_positions[(past_positions['asset'] == asset) & (past_positions['rt'] < reference_time)]
    asset_rows = asset_rows.sort_values(by='rt', ascending=False)
    nearest_date = asset_rows.iloc[0]['rt'].date()
    nearest_day_rows = asset_rows[asset_rows['rt'].dt.date == nearest_date]

    return nearest_day_rows['target_position'].sum()


In [11]:
today_positions = tpos[tpos['rt'] == trading_session]
today_positions["last_position"] = today_positions['asset'].apply(lambda asset: find_last_position(asset, tpos, trading_session))
today_positions["order_local"] = today_positions["target_position"] - today_positions["last_position"]

/var/folders/80/qjcx023s5dsfl5g98wkn22yc0000gn/T/ipykernel_10193/3656953785.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  today_positions["last_position"] = today_positions['asset'].apply(lambda asset: find_last_position(asset, tpos, trading_session))
/var/folders/80/qjcx023s5dsfl5g98wkn22yc0000gn/T/ipykernel_10193/3656953785.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  today_positions["order_local"] = today_positions["target_position"] - today_positions["last_position"]


In [12]:
print("Orders by strategy:")
today_positions[["strategy", "asset", "order_local"]]

Orders by strategy:


,strategy,asset,order_local
3,s1,CZK,-66000000.0
7,s2,AUD,500000.0
11,s3,CZK,6000000.0


In [13]:
print("Total orders:")
today_positions.groupby('asset', as_index=False)['order_local'].sum()

Total orders:


,asset,order_local
0,AUD,500000.0
1,CZK,-60000000.0


### Generate orders (total & by strategy) as of trading session in USD

In [14]:
todays_fx = fx_rates[fx_rates['rt'] == trading_session]

today_positions = today_positions.merge(todays_fx, on=['asset', 'rt'], how='left')
today_positions['order_usd'] = today_positions['order_local'] / today_positions['fx_rate']

In [15]:
print("Orders by strategy in USD:")
today_positions[['strategy', 'asset', 'order_usd']]

Orders by strategy in USD:


,strategy,asset,order_usd
0,s1,CZK,-2.868318e+06
1,s2,AUD,3.450000e+05
2,s3,CZK,2.607562e+05


In [16]:
print("Total orders in USD:")
total_orders = today_positions.groupby('asset')['order_usd'].sum().to_frame().reset_index()
total_orders

Total orders in USD:


,asset,order_usd
0,AUD,3.450000e+05
1,CZK,-2.607562e+06


### Apply Minimum Order Size and generate new target position after this session

*Context: In practice there is often minimum cost we need to pay when trading. This means the orders must be of certain size to make economic sense. Therefore, if an order is below the limit size it will not be executed and this needs to be fed back to the target position, so it trades knowing that the past position is T-2, not T-1*

In [17]:
total_orders['validate_orders_USD'] = total_orders.apply(
    lambda row : row['order_usd'] if abs(row['order_usd']) >= min_order_size_usd[min_order_size_usd['asset'] == row['asset']]['min_order_size_usd'].iloc[0]
    else 0,
    axis=1
)

total_orders

,asset,order_usd,validate_orders_USD
0,AUD,3.450000e+05,0.000000e+00
1,CZK,-2.607562e+06,-2.607562e+06


In [18]:
# the idea here could be to replace the target position values for strategies which were too small to get executed today with yesterday's target position values, so the last position will be correct
# even easier actually would be to just remove today's unused tpos, though then you lose the decision for the record
# it probably makes more sense to have separate systems tracking target and actual positions though, rather than using them as one and the same
# in general could split out some things here into different reusable functions to make it cleaner (checking for inverted fx rates, database prep etc.)